In [32]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


import regex as re
import nltk

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

import time

In [1]:
url_nsq = 'https://www.reddit.com/r/NoStupidQuestions.json'
url_tifu = 'https://www.reddit.com/r/tifu.json' 

In [34]:
nsq_df = pd.read_csv('nsq.csv')
tifu_df = pd.read_csv('tifu.csv')


In [35]:
print(nsq_df.shape)
print(tifu_df.shape)

(133, 5)
(62, 5)


In [6]:
header = {'User-agent': 'This is only a test 1.3'}

In [7]:
def get_posts(red):
    posts = []
    for i in range(0,len(red['data']['children'])):
        post = {}
        post['title'] = red['data']['children'][i]['data']['title']
        post['selftext'] = red['data']['children'][i]['data']['selftext']
        post['subreddit'] = red['data']['children'][i]['data']['subreddit']
        post['author'] = red['data']['children'][i]['data']['author']
        post['id'] = red['data']['children'][i]['data']['id']

        posts.append(post)
    
    return pd.DataFrame(posts)

In [ ]:
for _ in range(1,51):
    req_nsq = requests.get(url_nsq, headers = header)
    req_tifu = requests.get(url_tifu, headers = header)
    nsq = get_posts(req_nsq.json())
    tifu = get_posts(req_tifu.json())
    nsq_df = pd.concat([nsq_df, nsq], axis = 'index')
    tifu_df = pd.concat([tifu_df, tifu], axis = 'index')
    time.sleep(120)
    print(_)

## I needed more posts for just one of them

In [14]:
for _ in range(1,61):
    req_nsq = requests.get(url_nsq, headers = header)
    nsq = get_posts(req_nsq.json())
    nsq_df = pd.concat([nsq_df, nsq], axis = 'index')
    time.sleep(60)
    print(_)

1
2
3
4
5
6
7
8
9
10
11
13
14
16
17
18
19
20
21
22
23
24
25
27
28
29
30
31
32
33
34
35
36
37
38
39
43
44
45
46
47
48
49
50
51
53
54
55
56
57
58
59
60


In [15]:
print(nsq_df.drop_duplicates().shape)
print(tifu_df.drop_duplicates().shape)

(1086, 6)
(1775, 6)


In [16]:
nsq_df.drop_duplicates(inplace = True)

In [17]:
tifu_df.drop_duplicates(inplace = True)

In [19]:
len(nsq_df) + len(tifu_df)

2861

In [23]:
tifu_df.sample(25)

,author,id,score,selftext,subreddit,title
347,lamullen,ca2oxv,22035,As per usual this didn't happen today but this...,tifu,"TIFU by diving into a pool headfirst, deglovin..."
853,GuyWithThermalShirts,ca9uwr,12,"This happened last night, we were about to get...",tifu,TIFU by making my wife as dry as the Gobi Dese...
494,cookielover45,caaoah,8,I actually posted this in another thread but I...,tifu,TIFU by making fun of my best friend
83,Llim,c3pm90,217,Hey everyone!\n\nHere's the voting thread for ...,tifu,Fuck Up of the Month Voting Thread | May 2019
1189,loserposerboboozer,ca7mz6,322,So this happened yesterday. I was horny and ho...,tifu,TIFU By asking my partner his biggest fantasy ...
816,Ragnadrok,ca5yhs,53,"I live at home with my parents, and just a bit...",tifu,TIFU by leaving my fleshlight in the shower.
891,youweretheclosest,ca1coa,889,First time posting so I hope at least someone ...,tifu,TIFU by announcing to my neighbors that I swallow
161,Llim,c3pm90,220,Hey everyone!\n\nHere's the voting thread for ...,tifu,Fuck Up of the Month Voting Thread | May 2019
625,ChuckleNut445,caat2s,438,"WARNING: NSFW (idk how to flair), this is gonn...",tifu,TIFU by trying anal with my wife
1247,getonthebusMFer,caal2f,125,This happened many years ago. Names/places cha...,tifu,TIFU by winning at trivial pursuit and making ...


In [24]:
tifu_df[tifu_df['title'] == 'TIFU by trying anal with my wife']

,author,id,score,selftext,subreddit,title
335,ChuckleNut445,caat2s,3,"WARNING: NSFW (idk how to flair), this is gonn...",tifu,TIFU by trying anal with my wife
356,ChuckleNut445,caat2s,5,"WARNING: NSFW (idk how to flair), this is gonn...",tifu,TIFU by trying anal with my wife
364,ChuckleNut445,caat2s,6,"WARNING: NSFW (idk how to flair), this is gonn...",tifu,TIFU by trying anal with my wife
373,ChuckleNut445,caat2s,7,"WARNING: NSFW (idk how to flair), this is gonn...",tifu,TIFU by trying anal with my wife
381,ChuckleNut445,caat2s,9,"WARNING: NSFW (idk how to flair), this is gonn...",tifu,TIFU by trying anal with my wife
388,ChuckleNut445,caat2s,10,"WARNING: NSFW (idk how to flair), this is gonn...",tifu,TIFU by trying anal with my wife
393,ChuckleNut445,caat2s,11,"WARNING: NSFW (idk how to flair), this is gonn...",tifu,TIFU by trying anal with my wife
404,ChuckleNut445,caat2s,29,"WARNING: NSFW (idk how to flair), this is gonn...",tifu,TIFU by trying anal with my wife
418,ChuckleNut445,caat2s,30,"WARNING: NSFW (idk how to flair), this is gonn...",tifu,TIFU by trying anal with my wife
429,ChuckleNut445,caat2s,34,"WARNING: NSFW (idk how to flair), this is gonn...",tifu,TIFU by trying anal with my wife


In [29]:
tifu_df = tifu_df.drop('score', axis = 'columns').drop_duplicates()

In [30]:
nsq_df = nsq_df.drop('score', axis = 'columns').drop_duplicates()

In [31]:
nsq_df.to_csv('nsq.csv', index = False)
tifu_df.to_csv("tifu.csv", index = False)